## Set-Up

In [14]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, inspect

In [15]:
# Create an engine for the `wine_db.sqlite` database
engine = create_engine("sqlite:///Resources/wine_data.db")

In [16]:
inspector = inspect(engine)
inspector.get_table_names()

['winequalityRed', 'winequalityWhite']

In [17]:
red_wine = engine.execute('SELECT * FROM winequalityRed').fetchall()

In [18]:
wine_red_df = pd.DataFrame(red_wine, columns=["wine_id","fixed acidity","volatile acidity", "citric acid",
                                "residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide",
                                "density","pH","sulphates","alcohol","quality"])

In [19]:
# Separate feature variables and target variable
X = wine_red_df.drop(['quality','wine_id'], axis = 1)
y = wine_red_df['quality']

In [20]:
# Normalize feature variables
from sklearn.preprocessing import StandardScaler
X_features = X
scalar = StandardScaler().fit(X)
X = StandardScaler().fit_transform(X)

In [21]:
# Splitting the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=0)

In [22]:
from sklearn.metrics import classification_report
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier(bootstrap=True,random_state=1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.72      0.76       151
           1       0.79      0.86      0.83       193

    accuracy                           0.80       344
   macro avg       0.80      0.79      0.79       344
weighted avg       0.80      0.80      0.79       344



In [23]:
import pickle
pkl_filename = "app/red_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model,file)

scaler = StandardScaler().fit(X)
pkl_scaled_filename = 'app/scaler_red.pkl'
with open(pkl_scaled_filename, 'wb') as file:
    pickle.dump(scaler,file)

In [24]:
# Function to Load and run model with input scores

def red_model(user_input_array):
    import pickle
    #Load Model
    with open('app/red_model.pkl', 'rb') as file:
        red_model = pickle.load(file)
    with open('app/scaler_red.pkl', 'rb') as file:
        scaler = pickle.load(file)

    X = user_input_array.reshape(1,-1)
    X_scaled = scaler.transform(X)

    # Calculate Wine quality
    y_pred_red = red_model.predict(X_scaled)

    return y_pred_red

In [27]:
test = X_test[0]
answer = y[0]
print(test, answer)

[-0.75398448  1.43194831 -0.94397982 -0.39415393  0.11909871 -0.52986225
 -0.87977495 -0.55525786  0.52891085 -1.17113651 -0.39666305] 0


In [28]:
red_model(test)

array([0])